<img src="http://oci02.img.iteso.mx/identidad_de_instancia_2018/ITESO/Logos%20ITESO/Logo-ITESO-Principal.jpg">

<center><font color= #004A94> <font size = 6>Msc Ciencia de datos</font> </font> <br> <br> <font color= #047CFB> <font size = 5>I.F. Juan Francisco Muñoz Elguezabal</font> <br> <br>

<center><font color= #42c29b> <font size = 6> Aplicación de modelo ANOVA en finanzas bursátiles</font> </font> <br> <br> <font color= #047CFB> <font size = 5>Repositorio: <a href='https://github.com/IFFranciscoME/FinTechLab/tree/master/anova'>Link</a></font>

---

## Abstract

En finanzas bursatiles ... 

## Introduccion

El modelo _ANOVA_, en Finanzas bursátiles, el calendario económico, el indicador economico, clasificacion de resultados, la reacción del precio.

### Cargar paqueterias y dependencias

In [5]:
import pandas as pd                                       # dataframes y utilidades
import numpy as np                                        # operaciones matematicas
import statsmodels.api as sm                              # modelos estadisticos: anova

from statsmodels.stats.multicomp import MultiComparison   # herramientas estadisticas: multicomparacion
from statsmodels.formula.api import ols                   # herramientas estadisticas: modelo lineal con ols
from datos import df_ce, df_pe                            # importar datos internamente de otro script

pd.set_option('display.max_rows', None)                   # sin limite de renglones maximos para mostrar pandas
pd.set_option('display.max_columns', None)                # sin limite de columnas maximas para mostrar pandas
pd.set_option('display.width', None)                      # sin limite el ancho del display
pd.set_option('display.expand_frame_repr', False)         # visualizar todas las columnas de un pandas dataframe
pd.options.mode.chained_assignment = None                 # para evitar el warning enfadoso de indexacion

### Adecuaciones iniciales

In [ ]:
# duplicar objetos para conservar los originales
df_data_pe = df_pe
df_data_ce = df_ce

# convertir a datetime columna de fechas
df_data_pe['timestamp'] = pd.to_datetime(df_data_pe['timestamp'])

# escenarios para clasificar los eventos
escenarios = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H']

### Definir metricas iniciales

Calcular metricas :

- **ho** (precio high - precio open): Como métrica para indicar que el precio cambio en una dirección _alcista_. 
- **ol** (precio open - precio low): Como métrica para indicar que el precio cambio en una dirección _bajista_.
- **hl** (precio high - precio low): Como métrica para indicar _volatilidad_ en el precio.

In [8]:
def f_reaccion(p0_i, p1_ad):
    """
    :param p0_i: indexador
    :param p1_ad: cantidad de precios hacia adelante que se considera la ventana

    :return: resultado: diccionario con resultado final con 3 elementos como resultado

    # debugging
    p0_i = 1
    p0_ad = 3
    """

    indice_1 = np.where(df_pe['timestamp'] == df_ce['timestamp'][p0_i])[0][0]
    indice_2 = indice_1 + p1_ad
    ho = round((max(df_pe['high'][indice_1:indice_2]) - df_pe['open'][indice_1])*10000, 2)
    ol = round((df_pe['open'][indice_1] - min(df_pe['low'][indice_1:indice_2]))*10000, 2)
    hl = round((max(df_pe['high'][indice_1:indice_2]) - min(df_pe['low'][indice_1:indice_2]))*10000, 2)

    # diccionario con resultado final
    resultado = {'ho': ho, 'ol': ol, 'hl': hl}

    return resultado

In [ ]:
# reaccion del precio para cada escenario
d_reaccion = [f_reaccion(p0_i=i, p1_ad=3) for i in range(0, len(df_ce['timestamp']))]

# acomodar resultados en columnas
df_data_ce['ho'] = [d_reaccion[j]['ho'] for j in range(0, len(df_data_ce['timestamp']))]
df_data_ce['ol'] = [d_reaccion[j]['ol'] for j in range(0, len(df_data_ce['timestamp']))]
df_data_ce['hl'] = [d_reaccion[j]['hl'] for j in range(0, len(df_data_ce['timestamp']))]

### Ocurrencia de escenarios

In [ ]:
# Escenarios que si ocurrieron
esc_reales = list(set(df_data_ce['escenario']))

# Elegir a escenarios con mayor similitud entre muestras
ocurrencias = dict(df_data_ce['escenario'].value_counts())

### Ajustar modelo ANOVA

In [ ]:
# ajustar modelo lineal
model = ols('ol ~ escenario', data=df_data_ce).fit()

# tabla anova
anova_table = sm.stats.anova_lm(model, typ=2)
print(anova_table)

### Prueba de Tukey

In [ ]:
# comparacion con tukey
mc = MultiComparison(df_data_ce['hl'], df_data_ce['escenario'])
mc_results = mc.tukeyhsd()
print(mc_results)

### Notas no ordenadas

se define como tiempo inicial como aquel en el que se comunico el indicador
proposito del ejercicio: probar si estadisticamente existe diferencia significativa en la media de las
reacciones del precio ante 8 diferentes escenarios de salida de un indicador economico

whether there is a statistically significant difference in mean systolic blood pressures among the four groups
h0 = todas las 'reacciones' del precio, ante la comunicacion de un mismo indicador economico, son iguales.